In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

In [ ]:
latest = pd.read_csv("../input/fake-review-dm/latest.csv", header = 0)

In [ ]:

latest.info()

In [ ]:
latest[latest['Label'] == 1].count()

In [ ]:
latest.head()

In [ ]:
latest.dropna(axis = 0, inplace=True)

In [ ]:
positive = latest[latest['Label']== 1]

In [ ]:
true_word_list = []
for i in range(0,positive['corpus'].count()):
   true_word_temp = positive['corpus'].iloc[i].split()
   true_word_list.append(true_word_temp)

In [ ]:
from itertools import chain

In [ ]:
list1 = list(chain.from_iterable(true_word_list))

In [ ]:
list1

In [ ]:

from collections import Counter 
true_mc = Counter(list1).most_common() 

In [ ]:
true_mc

In [ ]:
fake = latest[latest['Label'] == -1]

In [ ]:
fake_word_list= fake['corpus'].str.split(expand=True).stack()
from collections import Counter 
fake_mc = Counter(fake_word_list).most_common() 
fake_mc

In [ ]:
common_word_list= set(true_mc) - (set(true_mc) - set(fake_mc))

In [ ]:
df_true = pd.DataFrame(true_mc)

In [ ]:
df_true.head()

In [ ]:
df_false = pd.DataFrame(fake_mc)

In [ ]:
df_false1 = df_false[0:23]
df_true1 = df_true[0:23]

In [ ]:
df_common = list(set(df_true1.iloc[:,0]) & set(df_false1.iloc[:,0]))

In [ ]:
df_common

In [ ]:
len(df_common)

CNN - Keras

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
sentences_train, sentences_test, y_train, y_test = train_test_split(latest['corpus'], latest['Label'], test_size=0.25, random_state=101)

In [ ]:
sentences_train.head()

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


In [ ]:
vocab_size

In [ ]:
sentences_train.head(2)

In [ ]:
len(X_train)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 50

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

In [ ]:
y_train.replace(to_replace= -1, value = 0, inplace=True) 

In [ ]:
y_test.replace(-1, 0, inplace=True)

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(15000, embedding_dim, input_length=100))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
y_train[0]

In [ ]:
model.fit(X_resampled, y_resampled,
                    epochs=3,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=100)
loss, accuracy = model.evaluate(X_resampled, y_resampled, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

CNN - LSTM

In [ ]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly

In [ ]:
model = Sequential()
model.add(Embedding(15000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(X_resampled,y_resampled,validation_data=(X_test,y_test), epochs=3) 

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
pred = (predictions > 0.5)

In [ ]:
pred[5]

In [ ]:
labels = (predictions > 0.5).astype(np.int)

In [ ]:
len(labels[labels == [0]])

In [ ]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, labels)

In [ ]:
cf = classification_report(y_test, labels)

In [ ]:
print(cm)

In [ ]:
print(cf)

In [ ]:
to_bool(pred[10])

Count Vectorizer

In [ ]:
vectorizer = CountVectorizer(stop_words= df_common, ngram_range=(2,3), max_features= 15000)
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

TFIDF (optional)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer1 = TfidfVectorizer(stop_words= df_common, ngram_range=(3,3), max_features= 20000)
vectorizer1.fit(sentences_train)

X_train1 = vectorizer1.transform(sentences_train)
X_test1  = vectorizer1.transform(sentences_test)

In [ ]:
X_train1

ROS

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

In [ ]:
y_train.describe()

ML Models: 

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_resampled, y_resampled)
score = classifier.score(X_test1, y_test)

print("Accuracy:", score)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
predictions = classifier.predict(X_test1)

In [ ]:
predictions

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
confusion_matrix(y_test, predictions)

ML - NLP

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer1 = TfidfVectorizer()
train_vectors = vectorizer1.fit_transform(sentences_train)
test_vectors = vectorizer1.transform(sentences_test)
print(train_vectors.shape, test_vectors.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:

predicted = clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, predicted)

In [ ]:
print(classification_report(y_test, predicted))

In [ ]:
print(classification_report(y_test, predicted))

Data Cleaning

In [ ]:

# library to clean data 
import re  
  
# Natural Language Tool Kit 
import nltk  
  
# to remove stopword 
from nltk.corpus import stopwords 
  
# for Stemming propose  
from nltk.stem.porter import PorterStemmer 
  
# Initialize empty array 
# to append clean text  
corpus = []  

In [ ]:
# 1000 (reviews) rows to clean 
for i in range(0,merged_final['Review_Text'].count()):  
      
    # column : "Review", row ith 
    review = re.sub('[^a-zA-Z]', ' ', merged_final['Review_Text'][i])  
     
    # convert all cases to lower cases 
    review = review.lower()  
      
    # split to array(default delimiter is " ") 
    review = review.split()  
      
    # creating PorterStemmer object to 
    # take main stem of each word 
    ps = PorterStemmer()  
      
    # loop for stemming each word 
    # in string array at ith row     
    review = [ps.stem(word) for word in review 
                if not word in set(stopwords.words('english'))]  
                  
    # rejoin all string array elements 
    # to create back into a string 
    review = ' '.join(review)   
      
    # append each string to create 
    # array of clean text  
    corpus.append(review)  
    

In [ ]:
len(corpus)

In [ ]:
corpus.head()

In [ ]:
merged_final['corpus'] = corpus

In [ ]:
merged_final.head()

In [ ]:
print(merged_final['Review_Text'].count())

In [ ]:
merged_final.head()

In [ ]:
merged_final.to_csv('latest.csv', header=True, index=None)

In [ ]:
merged_final.to_csv('latest_tsv.tsv', header=True, index=None, sep = '\t')

In [ ]:
, 

Using K-fold - imbalanced data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
lr = LogisticRegression()

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              negative     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))


In [ ]:
from sklearn.pipeline import Pipeline
original_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', lr)
])
lr_cv(5, latest['Review_Text'], latest['Label'].reindex, original_pipeline, 'macro')

In [ ]:
latest['Label'].describe()

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777),lr)
SMOTE_pipeline = make_pipeline(tvec, SMOTE(random_state=777),lr)

In [ ]:
sentences_train = sentences_train.values.reshape(-1,1)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))


In [ ]:
X_resampled

We can see that the data is right skewed, Lets the the average ratings per restaurant

Adding feature in existing dataset of average rating of the product.

In [ ]:
#average_rating_per_product = average_rating_per_product.rename(columns={'Rating':'Avg_Rating'})

In [ ]:
#average_rating_per_product.head()

In [ ]:
#merged_final = merged_final.merge(average_rating_per_product, left_on='Product Name', right_on='Product Name')

In [ ]:
#merged_final.info()

In [ ]:
#merged_final = merged_final.drop(columns='Unnamed: 0')

In [ ]:
#merged_final.head()




Lets add feature of average rating that the user gives for comparison

In [ ]:
#average_rating_per_user = merged_final.groupby('User_Id')['Rating'].mean()

In [ ]:
#average_rating_per_user = average_rating_per_user.to_frame()

In [ ]:
#average_rating_per_user.reset_index(level=0, inplace=True)

In [ ]:
#average_rating_per_user.head()

In [ ]:
#average_rating_per_user = average_rating_per_user.rename(columns={'Rating':'Avg_user_rating'})

In [ ]:
#average_rating_per_user.head()

In [ ]:
#merged_final = merged_final.merge(average_rating_per_user, left_on='User_Id', right_on='User_Id')

In [ ]:
#merged_final.head()

In [ ]:
#merged_final.to_csv('final_merged.csv', header=True)

In [ ]:
#merged_final.head(10)

In [ ]:
sns.countplot(x = 'Rating',hue =  'Label', data = merged_final)

In [ ]:
#fake_review_data = merged_final[merged_final['Label'] == -1]

In [ ]:
#review=merged_final['Review_Text']
#y=merged_final['Label']

In [ ]:

#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer()


In [ ]:
#from sklearn.model_selection import train_test_split
#data_train, data_test, labels_train, labels_test = train_test_split(review, y, test_size=0.3)